# Convert h5 model to trt model

In [1]:
import sys 
import os

NOMEROFF_NET_DIR = "../../"

# setup result model dirs
SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/saved_model_ocr_eu_ua_1995")
TRT_SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/trt_saved_model_ocr_eu_ua_1995")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
import tensorflow as tf
import matplotlib.image as mpimg

# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

from NomeroffNet import  TextDetector

## load latest h5 model

In [3]:
textDetector = TextDetector.get_static_module("eu_ua_1995")()
textDetector.load("latest")

In [4]:
textDetector.MODEL.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input_eu_ua_1995 (InputLaye [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input_eu_ua_1995[0][0]       
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
_______________________________________________________________________________________

## convert to pb model

In [5]:
tf.saved_model.save(textDetector.MODEL, SAVED_MODEL_DIR)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../.././models/saved_model_ocr_eu_ua_1995/assets


In [6]:
1

1

## convert pb model to trt model

In [7]:
params = tf.experimental.tensorrt.ConversionParams(
    minimum_segment_size=50,
    max_workspace_size_bytes=1 << 25,
    precision_mode  = "FP16",
)

converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=SAVED_MODEL_DIR,
    conversion_params=params)
converter.convert()
converter.save(TRT_SAVED_MODEL_DIR)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: ../.././models/trt_saved_model_ocr_eu_ua_1995/assets


## run trt model

In [8]:
saved_model_loaded = tf.saved_model.load(TRT_SAVED_MODEL_DIR)

In [9]:
import matplotlib.image as mpimg
import numpy as np

img_path = '../../examples/crop_np_images/JJF509.png'
imgs = [mpimg.imread(img_path)]
Xs = []
for img in imgs:
    Xs.append(textDetector.normalize_pb(img[:,:,:3]))

In [10]:
f = saved_model_loaded.signatures["serving_default"]

In [11]:
tensorX = tf.convert_to_tensor(np.array(Xs).astype(np.float32))

In [12]:
f(tensorX)

{'softmax_eu_ua_1995': <tf.Tensor: shape=(1, 32, 23), dtype=float32, numpy=
 array([[[1.72328422e-04, 4.38883379e-02, 1.17942509e-04, 2.91991848e-02,
          7.98262190e-03, 1.90976512e-04, 2.25845088e-05, 6.89068518e-04,
          1.69387341e-01, 3.51662159e-01, 8.89563747e-03, 7.96942273e-04,
          1.46383494e-02, 1.98502228e-01, 3.30648647e-04, 4.30046879e-02,
          4.06672807e-05, 5.42280788e-04, 1.04214419e-02, 6.76679052e-03,
          3.98287065e-02, 3.44933732e-03, 6.94697499e-02],
         [8.48365926e-06, 1.42831483e-03, 6.34162632e-07, 2.52763764e-03,
          6.35214092e-05, 1.00704931e-07, 2.16316494e-06, 6.14353576e-06,
          1.39174625e-01, 6.33008638e-03, 1.02696847e-03, 5.19496143e-06,
          2.72678095e-04, 5.76938364e-05, 5.05877597e-06, 5.56068844e-05,
          1.12862608e-05, 1.26361977e-06, 6.10941788e-05, 5.95328966e-05,
          1.59880088e-04, 2.60193119e-05, 8.48716080e-01],
         [2.01105022e-09, 1.43343621e-08, 7.16859211e-11, 1.300105